## **Preentrenamiento de LLMs con Hugging Face**


### Configuración


### Instalación de librerías requeridas

Las siguientes librerías están **preinstaladas** en el entorno del curso. Sin embargo, si ejecutas estos comandos en otro entorno de Jupyter (por ejemplo, Watson Studio o Anaconda), deberás quitar el `#` antes de `!pip` en las celdas de código para instalarlas:

*PS: Para ejecutar este cuaderno en tu propio entorno, ten en cuenta que las versiones de las librerías pueden variar según las dependencias.*


In [ ]:
# Todas las bibliotecas necesarias para este laboratorio están listadas a continuación. 
# Las bibliotecas preinstaladas  están comentadas.
# !pip install -qy pandas==1.3.4 numpy==1.21.4 seaborn==0.9.0 matplotlib==3.5.0 torch==2.1.0+cu118
# - Actualizar un paquete específico
# !pip install pmdarima -U
# - Actualar un paquete a una versión concreta
# !pip install --upgrade pmdarima==2.0.2
# Nota: Si tu entorno no soporta "!pip install", usa "!mamba install"

Las siguientes librerías **no** están preinstaladas. **Debes ejecutar la siguiente celda** para instalarlas:



In [ ]:
#!pip install transformers==4.40.0 
#!pip install -U git+https://github.com/huggingface/transformers
#!pip install datasets # 2.15.0
#!pip install portalocker>=2/0.0
#!pip install -q -U git+https://github.com/huggingface/accelerate.git
#!pip install torch==2.3.0
#!pip install -U torchvision
#!pip install protobuf==3.20.*


### Importación de librerías requeridas

*Se recomienda importar todas las librerías necesarias en un solo lugar (aquí):*

* **Nota**: si obtienes un error tras ejecutar la celda, intenta reiniciar el kernel; algunos paquetes necesitan reinicio para surtir efecto.


In [ ]:
import torch
from torch.optim.lr_scheduler import LambdaLR
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import AutoConfig,AutoModelForCausalLM,AutoModelForSequenceClassification,BertConfig,BertForMaskedLM,TrainingArguments, Trainer, TrainingArguments
from transformers import AutoTokenizer,BertTokenizerFast,TextDataset,DataCollatorForLanguageModeling
from transformers import pipeline
from datasets import load_dataset

from tqdm.auto import tqdm
import math
import time
import os


# Sección para suprimir advertencias generadas por el código:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

Desactiva el paralelismo de los tokenizadores para evitar bloqueos:


In [ ]:
# Establece la variable de entorno TOKENIZERS_PARALLELISM a 'false'
os.environ['TOKENIZERS_PARALLELISM'] = 'false'


### **Preentrenamiento y Fine-Tuning auto-supervisado**


El preentrenamiento es una técnica en procesamiento de lenguaje natural (NLP) que entrena LLMs en un gran corpus de texto no etiquetado. El objetivo es capturar patrones generales y relaciones semánticas del lenguaje natural, permitiendo al modelo comprender en profundidad la estructura y significado del lenguaje.

La motivación de preentrenar transformers es superar las limitaciones de enfoques tradicionales de NLP, que requieren muchos datos etiquetados para cada tarea. Al aprovechar la abundancia de texto no etiquetado, el preentrenamiento permite al modelo aprender habilidades lingüísticas fundamentales mediante objetivos auto-supervisados, facilitando el aprendizaje por transferencia.

Objetivos como el *masked language modeling* (MLM) y la *next sentence prediction* (NSP) son clave en el éxito de los transformers. Los modelos preentrenados pueden afinarse (fine-tuning) con datos sin etiquetar de un dominio específico (self-supervised fine-tuning) o con datos etiquetados para tareas concretas (supervised fine-tuning), mejorando aún más su rendimiento.

En las siguientes secciones, explorarás los objetivos de preentrenamiento, cómo cargar modelos preentrenados, la preparación de datos y el proceso de fine-tuning. Al finalizar, comprenderás a fondo el preentrenamiento y el fine-tuning auto-supervisado, y estarás listo para aplicar estas técnicas en problemas reales de NLP.


Comencemos cargando un modelo preentrenado de Hugging Face y realizando una inferencia:


In [ ]:
modelo = AutoModelForCausalLM.from_pretrained("facebook/opt-350m")
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")

pipe = pipeline("text-generation", model=modelo,tokenizer=tokenizer)
print(pipe("This movie was really")[0]["generated_text"])

### **Objetivos de preentrenamiento**

Los objetivos de preentrenamiento definen las tareas en las que el modelo se entrena durante esta fase, permitiéndole aprender representaciones contextuales profundas. Tres objetivos comunes son:

1. **Masked Language Modeling (MLM)**
   Consiste en enmascarar aleatoriamente algunas palabras en una oración y entrenar al modelo para predecirlas según el contexto circundante. El objetivo es que el modelo aprenda comprensión contextual y rellene la información faltante.

2. **Next Sentence Prediction (NSP)**
   Entrena al modelo para determinar si dos oraciones son consecutivas en el texto original o han sido emparejadas aleatoriamente. Ayuda a captar relaciones a nivel de oración y coherencia entre ellas.

3. **Predicción del Siguiente Token (Next Token Prediction)**
   El modelo recibe una secuencia de texto y aprende a predecir cuál es el siguiente token más probable basándose en el contexto anterior.

Diferentes modelos pueden usar variaciones o combinaciones de estos objetivos según su arquitectura y configuración de entrenamiento.


### **Entrenamiento auto-supervisado de un modelo BERT**

Entrenar un modelo BERT es un proceso complejo y que requiere un gran corpus de texto sin etiquetar y recursos computacionales significativos. A continuación te presentamos un ejercicio simplificado para ilustrar los pasos involucrados en el preentrenamiento de un modelo BERT usando el objetivo de MLM (Masked Language Modeling).

En este ejercicio utilizaremos la biblioteca Hugging Face Transformers, que provee modelos BERT ya implementados y herramientas para el preentrenamiento. Se te indicará que realices las siguientes tareas:

* Preparar el conjunto de datos de entrenamiento
* Entrenar un tokenizador
* Preprocesar el conjunto de datos
* Preentrenar BERT usando una tarea MLM
* Evaluar el modelo entrenado

#### **Importación de los conjuntos de datos necesarios**

El conjunto de datos WikiText es un benchmark muy utilizado en procesamiento de lenguaje natural (NLP). Contiene texto extraído de artículos de Wikipedia, limpiado para eliminar formato, enlaces y metadatos, resultando en un corpus de texto "crudo".

WikiText ofrece 4 configuraciones diferentes y se divide en tres partes: entrenamiento, validación y prueba. El conjunto de entrenamiento sirve para entrenar los modelos de lenguaje, mientras que los de validación y prueba se utilizan para evaluar su desempeño. Primero, carguemos los datos y concatenémoslos para crear un único conjunto de datos.

*Nota: El BERT original se preentrenó sobre los conjuntos de datos de Wikipedia y BookCorpus.*


In [ ]:
# Carga el conjunto de datos
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

Veamos la estructura del conjunto de datos:


In [ ]:
print(dataset)

Revisemos un registro de ejemplo:


In [ ]:
#Revisa un registro de ejemplo
dataset["train"][400]

Este conjunto contiene 36 718 filas de datos de entrenamiento. Si no dispones de un entorno con GPU, tal vez necesites reducir el tamaño del dataset. Puedes descomentar las siguientes líneas para seleccionar solo una parte:


In [ ]:
#dataset["train"] = dataset["train"].select([i for i in range(1000)])
#dataset["test"] = dataset["test"].select([i for i in range(200)])

A continuación, guardamos los textos en archivos de texto para crear objetos `TextDataset`:


In [ ]:
# Rutas para guardar los datasets en archivos de texto
output_file_train = "wikitext_dataset_train.txt"
output_file_test  = "wikitext_dataset_test.txt"

# Guarda el conjunto de entrenamiento en un archivo de texto
with open(output_file_train, "w", encoding="utf-8") as f:
    for example in dataset["train"]:
        # Escribe cada texto en una nueva línea
        f.write(example["text"] + "\n")

# Guarda el conjunto de prueba en un archivo de texto
with open(output_file_test, "w", encoding="utf-8") as f:
    for example in dataset["test"]:
        # Escribe cada texto en una nueva línea
        f.write(example["text"] + "\n")

Debes definir un tokenizador para convertir tu texto en tokens numéricos:


In [ ]:
# Crea un tokenizador BERT reutilizando tokens especiales de uno preentrenado
bert_tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

In [ ]:
model_name = 'bert-base-uncased'

modelo = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, is_decoder=True)


### **Entrenamiento de un tokenizador (opcional)**

En la celda anterior creaste una instancia de un tokenizador a partir de un tokenizador BERT preentrenado. Si quieres entrenar el tokenizador con tu propio conjunto de datos, puedes descomentar el código que aparece a continuación. Esto es especialmente útil cuando usas Transformers en áreas específicas, como la medicina, donde los tokens son de algún modo diferentes a los tokens generales en los que se basan los tokenizadores preexistentes. 

(Puedes omitir este paso si no deseas entrenar el tokenizador con tus datos específicos).



In [ ]:
## crea un generador de Python para cargar los datos de forma dinámica
def batch_iterator(batch_size=10000):
    for i in tqdm(range(0, len(dataset), batch_size)):
        yield dataset['train'][i : i + batch_size]["text"]

## crea un tokenizador a partir de uno existente para reutilizar los tokens especiales
bert_tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

## entrena el tokenizador usando nuestro propio conjunto de datos
bert_tokenizer = bert_tokenizer.train_new_from_iterator(text_iterator=batch_iterator(), vocab_size=30522)

### **Preentrenamiento**

En este paso, definimos la configuración del modelo BERT y creamos el modelo:

#### Definir la configuración de BERT

Aquí definimos los parámetros de configuración de un modelo BERT usando `BertConfig`. Esto incluye ajustar varios parámetros relacionados con la arquitectura del modelo:

* **vocab\_size=30522**: Especifica el tamaño del vocabulario. Este número debe coincidir con el tamaño de vocabulario usado por el tokenizador.
* **hidden\_size=768**: Establece el tamaño de las capas ocultas.
* **num\_hidden\_layers=12**: Determina el número de capas ocultas en el modelo Transformer.
* **num\_attention\_heads=12**: Establece el número de cabeceras de atención en cada capa de atención.


In [ ]:
# Define la configuración de BERT
config = BertConfig(
    vocab_size=len(bert_tokenizer.get_vocab()),  # Especifica el tamaño del vocabulario (asegúrate de que este número sea igual al vocab_size del tokenizador)
    hidden_size=768,                            # Establece el tamaño de las capas ocultas
    num_hidden_layers=12,                       # Establece el número de capas ocultas
    num_attention_heads=12,                     # Establece el número de cabeceras de atención
    intermediate_size=3072,                     # Establece el tamaño de la capa intermedia
)

Crea el modelo BERT para preentrenamiento


In [ ]:
# Crea el modelo BERT para preentrenamiento
modelo = BertForMaskedLM(config)

Verifica la configuración del modelo


In [ ]:
#Verifica la configuración del modelo
modelo

#### **Definición del conjunto de datos de entrenamiento**

Aquí definimos un conjunto de datos de entrenamiento usando la clase `TextDataset`, que sirve para cargar y procesar texto para entrenar modelos de lenguaje. Esta configuración típicamente implica:

* **tokenizer=bert\_tokenizer**: Especifica el tokenizador a usar. `bert_tokenizer` convierte el texto en tokens comprensibles por el modelo.
* **file\_path="wikitext\_dataset\_train.txt"**: Ruta al archivo de datos de preentrenamiento.
* **block\_size=128**: Define la longitud de las secuencias en las que el modelo será entrenado.

La clase `TextDataset` está diseñada para tomar grandes fragmentos de texto (como los que se encuentran en el archivo especificado), tokenizarlos y procesarlos de manera eficiente en bloques manejables del tamaño indicado.



In [ ]:
# Prepara los datos de preentrenamiento como un TextDataset
train_dataset = TextDataset(
    tokenizer=bert_tokenizer,
    file_path="wikitext_dataset_train.txt",  # Ruta al archivo de datos de preentrenamiento
    block_size=128                           # Establece el tamaño de bloque deseado para el entrenamiento
)
test_dataset = TextDataset(
    tokenizer=bert_tokenizer,
    file_path="wikitext_dataset_test.txt",   # Ruta al archivo de datos de prueba
    block_size=128                           # Establece el tamaño de bloque deseado para el entrenamiento
)

Al examinar una muestra, los índices de los tokens se muestran aquí con el tamaño de bloque:


In [ ]:
train_dataset[0]

Luego, preparamos los datos para la tarea MLM (enmascaramiento de tokens aleatorios):

#### **Definir el data collator para modelado de lenguaje**

Esta línea de código configura un `DataCollatorForLanguageModeling` de la librería Hugging Face Transformers. Un *data collator* se utiliza durante el entrenamiento para crear lotes de datos de forma dinámica. 

Para el modelado de lenguaje, especialmente para modelos como BERT que emplean *masked language modeling* (MLM), este collator prepara los lotes de entrenamiento enmascarando automáticamente tokens según una probabilidad especificada. A continuación, los detalles de los parámetros utilizados:

* **tokenizer=bert\_tokenizer**: Especifica el tokenizador que usará el data collator. El `bert_tokenizer` se encarga de tokenizar el texto y convertirlo al formato que espera el modelo.
* **mlm=True**: Indica que el data collator debe enmascarar tokens para el entrenamiento de masked language modeling. Al activarse, el collator enmascara aleatoriamente algunos tokens de los datos de entrada, los cuales el modelo intentará predecir.
* **mlm\_probability=0.15**: Establece la probabilidad con la que se enmascararán los tokens. Una probabilidad de 0.15 significa que, en promedio, el 15 % de los tokens de cada secuencia serán reemplazados por el token de máscara.


In [ ]:
# Prepara el data collator para modelado de lenguaje
data_collator = DataCollatorForLanguageModeling(
    tokenizer=bert_tokenizer,
    mlm=True,
    mlm_probability=0.15
)

In [ ]:
# Verifica cómo el collator transforma un registro de datos de ejemplo
data_collator([train_dataset[0]])

Ahora entrenamos el modelo BERT usando el módulo `Trainer`. (Para ver la lista completa de argumentos de entrenamiento, consulta [aquí](https://huggingface.co/docs/transformers/v4.33.2/en/main_classes/trainer#transformers.TrainingArguments)):

Esta sección configura el proceso de entrenamiento indicando diversos parámetros que controlan cómo se entrena, evalúa y guarda el modelo:

* **output\_dir="./trained\_model"**: Especifica el directorio donde se guardará el modelo entrenado y otros archivos de salida.
* **overwrite\_output\_dir=True**: Si se establece en `True`, sobrescribirá el contenido del directorio de salida si ya existe. Esto resulta útil al ejecutar experimentos varias veces.
* **do\_eval=True**: Habilita la evaluación del modelo. Si es `True`, el modelo se evaluará en los intervalos especificados.
* **evaluation\_strategy="epoch"**: Define cuándo debe evaluarse el modelo. Al ponerlo en `"epoch"`, se evaluará al final de cada época.
* **learning\_rate=5e-5**: Establece la tasa de aprendizaje para entrenar el modelo. Es un valor típico para ajustar modelos tipo BERT.
* **num\_train\_epochs=10**: Especifica el número de épocas de entrenamiento. Cada época corresponde a un pase completo sobre los datos de entrenamiento.
* **per\_device\_train\_batch\_size=2**: Fija el tamaño de lote para el entrenamiento en cada dispositivo. Debe ajustarse según la memoria disponible de tu hardware.
* **save\_total\_limit=2**: Limita el número total de puntos de control (checkpoints) que se guardarán. Solo se conservarán los dos más recientes.
* **logging\_steps=20**: Determina cada cuántos pasos de entrenamiento se registrará información, lo cual ayuda a supervisar el proceso.


In [ ]:
'''# Define los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir="./trained_model",      # Especificar el directorio de salida para el modelo entrenado
    overwrite_output_dir=True,         # Sobrescribir el contenido del directorio de salida si ya existe
    do_eval=True,                      # Realizar evaluación durante el entrenamiento
    evaluation_strategy="epoch",       # Estrategia de evaluación: al final de cada época
    learning_rate=5e-5,                # Tasa de aprendizaje
    num_train_epochs=10,               # Especificar el número de épocas de entrenamiento
    per_device_train_batch_size=2,     # Tamaño de lote por dispositivo durante el entrenamiento
    save_total_limit=2,                # Límite máximo de puntos de control guardados
    logging_steps=20                   # Registrar información cada 20 pasos
)

# Instancia el Trainer
trainer = Trainer(
    model=modelo,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Inicia el preentrenamiento
trainer.train()
'''

#### **Evaluación del rendimiento del modelo**

Comprobemos cómo se desempeña el modelo entrenado. La **perplejidad** (perplexity) se usa comúnmente para comparar diferentes modelos de lenguaje o configuraciones de un mismo modelo. Después del entrenamiento, la perplejidad se puede calcular sobre un conjunto de evaluación reservado para medir el rendimiento. Se calcula alimentando el conjunto de evaluación al modelo y comparando las probabilidades predichas de los tokens objetivo con los valores reales de los tokens enmascarados.

Una puntuación de perplejidad más baja indica que el modelo entiende mejor el lenguaje y es más eficaz prediciendo los tokens enmascarados. Esto sugiere que el modelo ha aprendido representaciones útiles y puede generalizar bien a datos no vistos.


In [ ]:
'''eval_results = trainer.evaluate()
print(f"Perplejidad: {math.exp(eval_results['eval_loss']):.2f}")'''

#### Carga del modelo guardado

Si deseas omitir el entrenamiento y cargar el modelo que entrenaste durante 10 épocas, descomenta la siguiente celda:

In [ ]:
#!wget 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/BeXRxFT2EyQAmBHvxVaMYQ/bert-scratch-model.pt'
#modelo.load_state_dict(torch.load('bert-scratch-model.pt',map_location=torch.device('cpu')))

La forma más sencilla de probar el modelo en modo inferencia es usarlo en un `pipeline()`. Instancia un pipeline para la tarea *fill-mask* con tu modelo y pásale el texto. Si lo deseas, puedes usar el parámetro `top_k` para especificar cuántas predicciones devolver:

In [ ]:
# Define el texto de entrada con un token enmascarado
text = "This is a [MASK] movie!"

# Crea un pipeline para la tarea "fill-mask"
mask_filler = pipeline("fill-mask", model=modelo, tokenizer=bert_tokenizer)

# Genera predicciones rellenando el token enmascarado
results = mask_filler(text)  # Se puede especificar top_k

# Imprime las secuencias predichas
for result in results:
    print(f"Token predicho: {result['token_str']}, Confianza: {result['score']:.2f}")

Verás que `[MASK]` se reemplaza por el token más frecuente. Este rendimiento limitado puede deberse a un entrenamiento insuficiente, falta de datos, arquitectura del modelo o a no ajustar correctamente los hiperparámetros. 

Probemos ahora con un modelo preentrenado de Hugging Face:


### **Inferencia con un modelo BERT preentrenado**


In [ ]:
# Cargar el modelo y tokenizador BERT preentrenados
pretrained_model = BertForMaskedLM.from_pretrained('bert-base-uncased')
pretrained_tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

# Definir el texto de entrada con un token enmascarado
text = "This is a [MASK] movie!"

# Crear el pipeline para "fill-mask"
mask_filler = pipeline(task='fill-mask', model=pretrained_model, tokenizer=pretrained_tokenizer)

# Realizar inferencia usando el pipeline
results = mask_filler(text)
for result in results:
    print(f"Token predicho: {result['token_str']}, Confianza: {result['score']:.2f}")

Este modelo preentrenado funciona mucho mejor que el modelo que entrenaste solo unas pocas épocas con un único conjunto de datos. Aun así, los modelos preentrenados no están diseñados para tareas específicas como extracción de sentimiento o clasificación de secuencias. Por eso se introducen métodos de **fine-tuning supervisado**.


### **Ejercicios**


1. Crea un modelo y un tokenizador usando la librería Hugging Face.
2. Visita este [enlace](https://huggingface.co/datasets?task_categories=task_categories:text-classification&sort=trending).
3. Elige un conjunto de datos de clasificación de texto que puedas cargar, por ejemplo `stanfordnlp/snli`.
4. Utiliza ese conjunto de datos para entrenar tu modelo (ten en cuenta los recursos disponibles para el entrenamiento) y evalúalo.

> **Nota:** El entorno del cuaderno no dispone de recursos suficientes para soportar entrenamientos pesados y esto podría causar que el kernel deje de funcionar.


In [ ]:
## Tu respuesta